#### Imports

In [38]:
from pyspark import SparkContext
from datetime import datetime

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Spark


Create a new SparkContext

In [39]:
sc = spark.sparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
threshold=1000
min_std_lift=0.2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Read data

Here we read the test data and map it by splitting by "," and ignoring the header row.

In [41]:
data = sc.textFile("s3://diomdle2021/conditions.csv")
data = data.map(lambda line: line.split(",")).filter(lambda v: v[0]!="START")
data.take(2)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['2017-01-14', '2017-03-30', '09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a', '88e540ab-a7d7-47de-93c1-720a06f3d601', '65363002', 'Otitis media'], ['2012-09-15', '2012-09-16', 'b0a03e8c-8d0f-4242-9548-40f4d294eba8', 'e89414dc-d0c6-478f-86c0-d08bac6ad0a2', '241929008', 'Acute allergic reaction']]

### Reorganize data

Next we map the unique conditions by their code and respective description

In [42]:
condition_map = data.map(lambda line: (int(line[4]),line[5])) \
.distinct()

condition_map.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(15777000, 'Prediabetes'), (40275004, 'Contact dermatitis'), (49436004, 'Atrial Fibrillation'), (126906006, 'Neoplasm of prostate'), (444470001, 'Injury of anterior cruciate ligament')]

And save the data  to new file

In [43]:
format_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
condition_map.saveAsTextFile("{0}/{1}".format("s3://diomdle2021/condition_map", format_time))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Here we get all the unique patient-condition pairs to a map.

We map by patient-(condition) code

In [44]:
#Map by PATIENT-(CONDITION)CODE, 
patient_rdd2 = data.map(lambda line: (line[2],line[4])) \
.groupByKey() \
.map(lambda tp: [int(j) for j in set(tp[1])]) 

patient_rdd2.cache()
n_buckets = patient_rdd2.count()
patient_rdd2.take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[65966004, 55822004, 162864005, 24079001], [156073000, 19169002, 55822004, 82423001, 15777000, 10509002, 40055000, 72892002, 444814009, 75498004]]

### Most frequent

Here we map the most frequent conditions and their counts.

flatMap: We flatmap them by key, and set their value to 1;
reduceByKey: We reduce by key and make all keys unique and their values their respective counts;
sortBy: We sort it by decreasingly by their count;
filter: And finally we apply the threshold, keeping only the ones that have higher counts than it. 

In [45]:
frequent_singles = patient_rdd2 \
.flatMap(lambda l: [(i,1) for i in l]) \
.reduceByKey(lambda v1,v2:v1+v2) \
.sortBy(lambda a:a[1],False) \
.filter(lambda k: k[1]>threshold)

frequent_singles.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(444814009, 751940), (195662009, 524692), (10509002, 461495), (162864005, 365567), (271737000, 355372), (15777000, 354315), (59621000, 305134), (40055000, 250239), (72892002, 205390), (19169002, 201894)]

Once we have the most frequent items we create a set with them

In [46]:
frequent_single_rdd = frequent_singles.map(lambda a: a[0])
frequent_single_set = sc.broadcast(set(frequent_single_rdd.collect()))
frequent_singles.saveAsTextFile("{0}/{1}".format("s3://diomdle2021/frequent_singles_count", format_time))
frequent_single_set.value

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{36971009, 196416002, 239872002, 399211009, 55680006, 90560007, 408512008, 84757009, 75498004, 201834006, 195967001, 192127007, 156073000, 263102004, 185086009, 301011002, 65363002, 92691004, 307731004, 40275004, 239720000, 254632001, 410429000, 39848009, 422034002, 197927001, 72892002, 47505003, 58150001, 38822007, 43878008, 65275009, 68496003, 198992004, 83664006, 59621000, 446096008, 128613002, 127013003, 88805009, 126906006, 429007001, 24079001, 235919008, 62564004, 271737000, 283385000, 230265002, 95417003, 110030002, 55822004, 363406005, 233678006, 109838007, 195662009, 444814009, 403192003, 6072007, 10509002, 47693006, 162573006, 79586000, 5602001, 237602007, 40055000, 230690007, 82423001, 403191005, 449868002, 15777000, 19169002, 232353008, 444470001, 403190006, 7200002, 444448004, 74400008, 254837009, 367498001, 80394007, 370143000, 35999006, 94260004, 87433001, 33737001, 359817006, 241929008, 97331000119101, 90781000119102, 67811000119102, 124171000119105, 368581000119106, 15

### Build Pairs

In [47]:
def pair_forming(l):
    pairs=[]
    l.sort()
    for i in range(0,len(l)):
        if l[i] in frequent_single_set.value:
            for j in range(i+1,len(l)):
                if  l[i]!=l[j] and l[j] in frequent_single_set.value:
                    pairs.append((l[i],l[j]))
    return pairs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Build Triples

In [48]:
def triple_forming(l):
    triples=[]
    pairs = pair_forming(l)
    for i,j in pairs:
        for k in l:
            if (i,j) in frequent_pair_set.value and (i,k) in frequent_pair_set.value and (j,k) in frequent_pair_set.value and k!=i and k!=j:#or (j,i) in frequent_pairs#prob unecessary
                  triples.append(tuple(sorted([i,j,k])))
    return triples

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Count the pairs

In [49]:
pair_count = patient_rdd2.flatMap(pair_forming).map(lambda p:(p,1)).reduceByKey(lambda v1,v2: v1+v2).filter(lambda tp:tp[1]>=threshold).sortBy(lambda a: a[1],False)
frequent_pair_set = sc.broadcast(set(pair_count.map(lambda a: a[0]).collect()))
pair_count.saveAsTextFile("{0}/{1}".format("s3://diomdle2021/frequent_pairs_count", format_time))

pair_count.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((195662009, 444814009), 343651), ((10509002, 444814009), 302516), ((15777000, 271737000), 289176), ((162864005, 444814009), 243812), ((271737000, 444814009), 236847), ((15777000, 444814009), 236320), ((10509002, 195662009), 211065), ((59621000, 444814009), 203450), ((162864005, 195662009), 167438), ((40055000, 444814009), 165530)]

### Count the triples

In [50]:
triple_count = patient_rdd2.flatMap(triple_forming).map(lambda p:(p,1)).reduceByKey(lambda v1,v2: v1+v2).filter(lambda tp:tp[1]>=threshold)
triple_count.sortBy(lambda a: a[1], False).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((15777000, 271737000, 444814009), 192819), ((10509002, 195662009, 444814009), 139174), ((15777000, 195662009, 271737000), 132583), ((10509002, 15777000, 271737000), 115510), ((162864005, 195662009, 444814009), 111860), ((195662009, 271737000, 444814009), 108560), ((15777000, 195662009, 444814009), 108083), ((15777000, 59621000, 271737000), 99818), ((10509002, 162864005, 444814009), 97384), ((10509002, 271737000, 444814009), 94793)]

In [51]:
###FREQUENT TRIPLE SET (PROBABLY NOT NEEDED)
#frequent_triples_dict = triple_count.collectAsMap()
#print(frequent_triples_dict.keys())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
min_std_lift = 0.2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Association Rules: Confidence


In [53]:
def confidence_pairs(single_1, single_2,count):
    den = single_counts_dict.value[single_1] #had  0 in case miss
    if den==0:
        return 0
    else:
        return count/den

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
def confidence_triples(pair,single, count):
    den = pair_counts_dict.value.get(pair,0)
    if den==0:
        return 0
    else:
        return count/den

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Association Rules: Interest

    interestI -> j = p(j|I) − p(j)
    interestI -> j = confidence(I -> j)−baskets containing j / baskets

high positive interest: presence of I indicates the presence of j.

high negative interest: presence of I discourages the presence of j

In [55]:
def interest_pairs(single_1, single_2,count):
    return confidence_pairs(single_1, single_2,count) - (single_counts_dict.value[single_2]/n_buckets)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
def interest_triples(pair,single,count):
    return confidence_triples(pair,single,count) - (single_counts_dict.value[single]/n_buckets)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Association rules: Lift

    Lift(I) -> j = confidence(I -> j)/P(j) =P(I, j)/P(I)*P(j)

Lift (also known as the observed/expected ratio) is a measure of the degree of dependence between I and j. 

A lift of 1 indicates that I and j are independent

In [57]:
def lift_pairs(single_1, single_2,count):
    return confidence_pairs(single_1, single_2,count)/(single_counts_dict.value[single_2]/n_buckets)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
def lift_triples(pair,single,count):
    return confidence_triples(pair,single,count)/(single_counts_dict.value[single]/n_buckets)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Association rules: Standardised lift

Standardised lift ranges from 0 to 1.

This facilitates setting a fixed threshold for selecting the rules

In [59]:
def std_lift_pairs(single_1, single_2,count):
    p_single_1 = single_counts_dict.value[single_1]/n_buckets
    p_single_2 = single_counts_dict.value[single_2]/n_buckets
    
    if p_single_1!=0 and p_single_2!=0:
        num = lift_pairs(single_1,single_2, count) - (max(p_single_1+p_single_2-1,1/n_buckets)/(p_single_1*p_single_2))
        den = (1/p_single_1*p_single_2) - (max(p_single_1+p_single_2-1,1/n_buckets)/(p_single_1*p_single_2))
        if den!=0:
            return num/den
        else:
            return 0
    else:
        return 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
def std_lift_triples(pair,single,count):
    p_pair = pair_counts_dict.value.get(pair,0)/n_buckets
    p_single = single_counts_dict.value[single]/n_buckets
    if p_pair!=0 and p_single!=0:
        num = lift_triples(pair,single, count) - (max(p_pair+p_single-1,1/n_buckets)/(p_pair*p_single))
        den = (1/p_pair*p_single) - (max(p_pair+p_single-1,1/n_buckets)/(p_pair*p_single))
        if den!=0:
            return num/den
        else:
            return 0
    else:
        return 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Breaking frequent itemsets into rules with respective counts

In [61]:
def break_doubles(pair):
    single_1 = pair[0][0]
    single_2 = pair[0][1]
    count = pair[1]
    return [((single_1,single_2),count),((single_2,single_1),count)]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
def break_triples(triple_count):
    
    triple=triple_count[0]
    count = triple_count[1]
    first_pair = (triple[0],triple[1])
    second_pair = (triple[0],triple[2])
    third_pair = (triple[1],triple[2])
    pairs = [first_pair, second_pair, third_pair]
    singles = [2,1,0]
    candidate_rules=[]
    for i in range(3):
        candidate_rules.append(((pairs[i],triple[singles[i]],count)))
    
    return candidate_rules

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Getting metrics for the rules in the format:
left side of the rule,right side,standard lift,lift,confidence,interest

In [63]:
single_counts_dict = sc.broadcast(frequent_singles.collectAsMap())
pair_counts_dict = sc.broadcast(pair_count.collectAsMap())
pair_counts_dict.value


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{(195662009, 444814009): 343651, (10509002, 444814009): 302516, (15777000, 271737000): 289176, (162864005, 444814009): 243812, (271737000, 444814009): 236847, (15777000, 444814009): 236320, (10509002, 195662009): 211065, (59621000, 444814009): 203450, (162864005, 195662009): 167438, (40055000, 444814009): 165530, (195662009, 271737000): 162925, (15777000, 195662009): 162227, (10509002, 162864005): 146077, (10509002, 271737000): 142085, (10509002, 15777000): 141106, (59621000, 195662009): 140139, (72892002, 444814009): 136986, (19169002, 444814009): 134867, (59621000, 271737000): 126551, (10509002, 59621000): 122910, (15777000, 59621000): 119842, (40055000, 195662009): 114470, (59621000, 162864005): 110177, (40055000, 162864005): 103517, (15777000, 162864005): 103371, (15777000, 40055000): 101659, (162864005, 271737000): 101419, (10509002, 40055000): 100115, (40055000, 271737000): 98993, (43878008, 444814009): 98873, (72892002, 195662009): 94716, (19169002, 162864005): 93672, (19169002,

In [64]:
def get_metrics_double(pair):
    single_1 = pair[0][0]
    single_2 = pair[0][1]
    count = pair[1]
    
    confidence = confidence_pairs(single_1, single_2,count)
    interest = interest_pairs(single_1, single_2,count)
    lift = lift_pairs(single_1, single_2,count)
    std_lift = std_lift_pairs(single_1, single_2,count)
    return ([single_1],single_2,std_lift,lift,confidence,interest)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
def get_metrics_triple(triple):
    pair = triple[0]
    single = triple[1]
    count = triple[2]
    confidence = confidence_triples(pair, single,count)
    interest = interest_triples(pair, single,count)
    lift = lift_triples(pair, single,count)
    std_lift = std_lift_triples(pair, single,count)
    return (list(pair),single,std_lift,lift,confidence,interest)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
possible_triple_rules = triple_count.flatMap(break_triples)
possible_triple_rules.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((15777000, 40055000), 82423001, 5866), ((15777000, 82423001), 40055000, 5866), ((40055000, 82423001), 15777000, 5866), ((15777000, 40055000), 444814009, 67378), ((15777000, 444814009), 40055000, 67378), ((40055000, 444814009), 15777000, 67378), ((15777000, 156073000), 444814009, 8025), ((15777000, 444814009), 156073000, 8025), ((156073000, 444814009), 15777000, 8025), ((40055000, 156073000), 444814009, 5185)]

In [67]:
resulting_triples = possible_triple_rules.map(get_metrics_triple).filter(lambda a: a[2]>=min_std_lift).sortBy(lambda a: a[2], False)
resulting_triples.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[([6072007, 236077008], 94260004, 779.6309564877665, 261.77702397105384, 0.4633649932157395, 0.4615949181106494), ([162573006, 254632001], 67811000119102, 667.1674697846876, 514.4791111111111, 0.9995555555555555, 0.9976127059160496), ([162573006, 67811000119102], 254632001, 666.3988023320313, 514.4791111111111, 1.0, 0.998056286487822), ([6072007, 271737000], 94260004, 587.7686507896054, 275.8673575158306, 0.48830594184576487, 0.4865358667406748), ([236077008, 271737000], 94260004, 587.7686507896054, 275.8673575158306, 0.48830594184576487, 0.4865358667406748), ([6072007, 444814009], 94260004, 516.2009653307674, 260.67124673649283, 0.46140768446106767, 0.4596376093559776), ([236077008, 444814009], 94260004, 516.2009653307674, 260.67124673649283, 0.46140768446106767, 0.4596376093559776), ([162573006, 271737000], 67811000119102, 481.3815106720477, 77.35631755505251, 0.15029169367534032, 0.14834884403583437), ([254632001, 271737000], 67811000119102, 481.3815106720477, 514.3909318208202, 0.9

In [68]:
candidate_double_rules = pair_count.flatMap(break_doubles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
resulting_pairs = candidate_double_rules.map(get_metrics_double).filter(lambda a: a[2]).sortBy(lambda a: a[2])
resulting_pairs.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[([195967001], 444814009, 0.002440400092434995, 1.0124421961931973, 0.6576626240352812, 0.008082206992110041), ([403192003], 444814009, 0.0026205638576483167, 1.013903448922831, 0.658611825192802, 0.009031408149630882), ([94260004], 444814009, 0.0027761598366961853, 1.0195416345081063, 0.662274280136652, 0.012693863093480862), ([67811000119102], 444814009, 0.0030771152172229836, 1.029497881117422, 0.6687416629613161, 0.019161245918144942), ([254632001], 444814009, 0.0030791625327367712, 1.0297245288483419, 0.6688888888888889, 0.019308471845717712), ([65275009], 444814009, 0.0036708367162214305, 1.0351509612377363, 0.6724137931034483, 0.022833376060277133), ([235919008], 444814009, 0.0036708367162214305, 1.0351509612377363, 0.6724137931034483, 0.022833376060277133), ([1501000119109], 444814009, 0.004141719284392613, 1.0266416731434154, 0.6668863261943987, 0.01730590915122754), ([67811000119102], 195662009, 0.004381380028862513, 1.0231532186150958, 0.4637616718541574, 0.0104945917956214)

Uniting both types of rules X -> Y and X,Y -> Z to a single rdd

In [70]:
final_rules = resulting_triples.union(resulting_pairs).sortBy(lambda a: a[2])
final_rules.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[([195967001], 444814009, 0.002440400092434995, 1.0124421961931973, 0.6576626240352812, 0.008082206992110041), ([403192003], 444814009, 0.0026205638576483167, 1.013903448922831, 0.658611825192802, 0.009031408149630882), ([94260004], 444814009, 0.0027761598366961853, 1.0195416345081063, 0.662274280136652, 0.012693863093480862), ([67811000119102], 444814009, 0.0030771152172229836, 1.029497881117422, 0.6687416629613161, 0.019161245918144942), ([254632001], 444814009, 0.0030791625327367712, 1.0297245288483419, 0.6688888888888889, 0.019308471845717712), ([65275009], 444814009, 0.0036708367162214305, 1.0351509612377363, 0.6724137931034483, 0.022833376060277133), ([235919008], 444814009, 0.0036708367162214305, 1.0351509612377363, 0.6724137931034483, 0.022833376060277133), ([1501000119109], 444814009, 0.004141719284392613, 1.0266416731434154, 0.6668863261943987, 0.01730590915122754), ([67811000119102], 195662009, 0.004381380028862513, 1.0231532186150958, 0.4637616718541574, 0.0104945917956214)

In [71]:
conds = sc.broadcast(condition_map.collectAsMap())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Function to translate condition ids to names

In [72]:
def translate_conditions(line):
    temp_list = list(line)
    left_side = line[0]
    right_side = line[1]
    translated_left = [conds.value[i] for i in left_side]
    translated_right = conds.value[right_side]
    temp_list[0] = translated_left
    temp_list[1] = translated_right
    return tuple(temp_list)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Translating condition ids to their name in the final rules rdd

In [73]:
final_rules = final_rules.map(translate_conditions).sortBy(lambda a: a[2],False)
final_rules.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(['Bleeding from anus', 'Protracted diarrhea'], 'Secondary malignant neoplasm of colon', 779.6309564877665, 261.77702397105384, 0.4633649932157395, 0.4615949181106494), (['Bleeding from anus'], 'Secondary malignant neoplasm of colon', 779.6309564877665, 261.77702397105384, 0.4633649932157395, 0.4615949181106494), (['Protracted diarrhea'], 'Secondary malignant neoplasm of colon', 779.6309564877665, 261.77702397105384, 0.4633649932157395, 0.4615949181106494), (['Suspected lung cancer (situation)', 'Small cell carcinoma of lung (disorder)'], 'Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)', 667.1674697846876, 514.4791111111111, 0.9995555555555555, 0.9976127059160496), (['Small cell carcinoma of lung (disorder)'], 'Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)', 667.1674697846876, 514.4791111111111, 0.9995555555555555, 0.9976127059160496), (['Suspected lung cancer (situation)'], 'Primary small cell malignant neoplasm of lung  TNM stage 1 (dis

Saving results to a folder and stopping spark context

In [74]:
#FILES ARE STRUCTURED AS left_rule_side,right_rule_side,std_lift,lift,confidence,interest
final_rules.saveAsTextFile("{0}/{1}".format("s3://diomdle2021/extracted_rules", format_time))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…